# Módulo#3 (**Transformación de datos**) - By Monica


In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# -----------------------------------------------------------------------------
from scipy.stats import chi2_contingency, ttest_ind, shapiro, kstest, poisson, chisquare, expon

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


## `Fase 1 --> Exploración y Limpieza`

> 📌 En esta fase, hago una exploración de los datos, (reviso el tipo de dato, que valores tiene cada columna y que significa para el cliente y para el análisis). Por otro lado, espero saber el total de registros reales, para esto se quitan todos los duplicados y se trabaja con los valores finales para empezar hacer una limpieza, según criterios de análisis.

## -->Exploración de la Data

#### DATA --> Customer-Flight-Activity

In [2]:
df_fly = pd.read_csv('Customer-Flight-Activity.csv')
df_fly.sample(5)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
228240,791852,2018,2,8,0,8,1176,126.36,0,0
123020,351937,2017,8,0,0,0,0,0.00,0,0
394744,979330,2018,11,0,0,0,0,0.00,0,0
3649,108857,2017,12,0,0,0,0,0.00,0,0
355578,135382,2018,10,6,0,6,666,66.00,0,0


| `Nombre columna English`         | `En Español`                        |
|--------------------------------|-----------------------------------|
| Loyalty Number                 | Número de fidelización            |
| Year                           | Año                               |
| Month                          | Mes                               |
| Flights Booked                 | Vuelos Reservados                 |
| Flights with Companions        | Vuelos con acompañantes           |
| Total Flights                  | Total de vuelos                   |
| Distance                       | Distanciarse                      |
| Points Accumulated             | Puntos acumulados                 |
| Points Redeemed                | Puntos canjeados                  |
| Dollar Cost Points Redeemed    | Puntos de costo en dólares canjeados |


In [3]:

# EXPLORAR LOS DATAFRAME EN GENERAL 

def exploracion_datos(df):
    print('Filas y Columnas del DATAFRAME \n')
    print(f"El número de filas que tenemos es de {df.shape[0]}.\nEl número de columnas es de {df.shape[1]}\n")
    print('____________________________________________________________\n')
    
    print(' Nombre de todas las Columnas del DATAFRAME: \n')
    print(df.columns)
    print('____________________________________________________________\n')
    
    print('INFORMACIÓN GENERAL DEL DATAFRAME \n')
    print(df.info())
    print('____________________________________________________________\n')
    
    print('Ver los NULOS del DataFrame \n')
    print(f'Nulos de todo el data: --> {df.isnull().sum().mean() * 100} \n')
    for columna in df.columns:
        cantidad_valores_nulos = df[columna].isnull().mean() * 100
        print(f'La columna {columna}: {cantidad_valores_nulos}')
    print('____________________________________________________________\n')
    
    print('Valores ÚNICOS por columna:\n')
    for columna in df.columns:
        cantidad_valores_unicos = df[columna].unique()
        print(f'La columna {columna}: {len(cantidad_valores_unicos)}')
        print(f'La columna {columna}: {cantidad_valores_unicos}')
        
    print('____________________________________________________________\n')
    
    print('Valores DUPLICADOS por columna es de:\n')
    for columna in df.columns:
        cantidad_duplicados = df[columna].duplicated().sum()
        print(f'La columna {columna}: {cantidad_duplicados}')
    print('____________________________________________________________\n')
  
    print('--> RESUMEN ESTADÍSTICO \n')
    print('<<< Variables Numéricas >>> \n')
    print(df.describe().T)
   

In [4]:
exploracion_datos(df_fly)

Filas y Columnas del DATAFRAME 

El número de filas que tenemos es de 405624.
El número de columnas es de 10

____________________________________________________________

 Nombre de todas las Columnas del DATAFRAME: 

Index(['Loyalty Number', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')
____________________________________________________________

INFORMACIÓN GENERAL DEL DATAFRAME 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  i

In [5]:
df_fly['Loyalty Number'].duplicated().sum()

388887

In [6]:
df_fly['Loyalty Number'].value_counts()

Loyalty Number
678205    72
989528    48
373638    48
684889    48
684881    48
          ..
428526    24
428536    24
428565    24
428590    24
999891    24
Name: count, Length: 16737, dtype: int64

In [7]:
# Identificar filas duplicadas exactas (todas las columnas iguales)
exact_duplicates = df_fly[df_fly.duplicated(keep=False)]

# Filtrar los duplicados que también tienen el mismo 'Loyalty Number'
exact_duplicates_sorted = exact_duplicates.sort_values(by=['Loyalty Number', 'Year', 'Month'])

# Mostrar el DataFrame resultante
exact_duplicates_sorted


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
16942,101902,2017,2,0,0,0,0,0.0,0,0
16943,101902,2017,2,0,0,0,0,0.0,0,0
33843,101902,2017,3,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
151972,992168,2017,9,0,0,0,0,0.0,0,0
304080,992168,2018,6,0,0,0,0,0.0,0,0
304081,992168,2018,6,0,0,0,0,0.0,0,0
354783,992168,2018,9,0,0,0,0,0.0,0,0


In [8]:
len(exact_duplicates_sorted['Loyalty Number'].unique())

159

In [9]:
valor = exact_duplicates_sorted.loc[exact_duplicates_sorted['Loyalty Number'] == 101902 ]
valor_sorted = valor.sort_values(by=['Loyalty Number', 'Year', 'Month'])
valor_sorted


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
16942,101902,2017,2,0,0,0,0,0.0,0,0
16943,101902,2017,2,0,0,0,0,0.0,0,0
33843,101902,2017,3,0,0,0,0,0.0,0,0
33844,101902,2017,3,0,0,0,0,0.0,0,0
101447,101902,2017,7,0,0,0,0,0.0,0,0
101448,101902,2017,7,0,0,0,0,0.0,0,0
185952,101902,2017,12,0,0,0,0,0.0,0,0
185953,101902,2017,12,0,0,0,0,0.0,0,0


In [10]:
# Eliminar duplicados exactos (todas las columnas iguales) y conservar solo la primera ocurrencia
df_unique = exact_duplicates_sorted.drop_duplicates(keep='first')

# Mostrar el DataFrame resultante
df_unique


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
16942,101902,2017,2,0,0,0,0,0.0,0,0
33843,101902,2017,3,0,0,0,0,0.0,0,0
101447,101902,2017,7,0,0,0,0,0.0,0,0
185952,101902,2017,12,0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
33664,992168,2017,2,0,0,0,0,0.0,0,0
101268,992168,2017,6,0,0,0,0,0.0,0,0
151971,992168,2017,9,0,0,0,0,0.0,0,0
304080,992168,2018,6,0,0,0,0,0.0,0,0


#### DATA --> Customer-Loyalty-History

In [11]:
df_loyal = pd.read_csv('Customer-Loyalty-History.csv')
df_loyal.sample(5)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
16608,414211,Canada,Ontario,Ottawa,K1F 2R2,Female,Bachelor,57841.0,Married,Star,28622.31,Standard,2014,1,NaN,NaN
10409,334454,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Married,Star,2419.31,Standard,2016,8,NaN,NaN
12215,235930,Canada,Saskatchewan,Regina,S6J 3G0,Male,Bachelor,86728.0,Single,Star,3360.44,Standard,2013,12,NaN,NaN
3805,133128,Canada,British Columbia,Victoria,V10 6T5,Male,Bachelor,33784.0,Married,Star,2588.86,2018 Promotion,2018,2,NaN,NaN
13140,557289,Canada,Ontario,Kingston,M9K 2P4,Male,Bachelor,85067.0,Married,Star,4871.15,Standard,2017,3,NaN,NaN


| `English`             | `Español`                |
|---------------------|-------------------------|
| Loyalty Number      | Número de lealtad       |
| Country             | País                    |
| Province            | Provincia               |
| City                | Ciudad                  |
| Postal Code         | Código Postal           |
| Gender              | Género                  |
| Education           | Educación               |
| Salary              | Salario                 |
| Marital Status      | Estado Civil            |
| Loyalty Card        | Tarjeta de Lealtad      |
| CLV                 | Valor del Cliente (CLV) |
| Enrollment Type     | Tipo de Inscripción     |
| Enrollment Year     | Año de Inscripción      |
| Enrollment Month    | Mes de Inscripción      |
| Cancellation Year   | Año de Cancelación      |
| Cancellation Month  | Mes de Cancelación      |


In [13]:

# EXPLORAR LOS DATAFRAME EN GENERAL 

def exploracion_datos1(df):
    print(' Filas y Columnas del DATAFRAME \n')
    print(f"El número de filas que tenemos es de {df.shape[0]}.\nEl número de columnas es de {df.shape[1]}\n")
    print('____________________________________________________________\n')
    
    print(' Nombre de todas las Columnas del DATAFRAME: \n')
    print(df.columns)
    print('____________________________________________________________\n')
    
    print('INFORMACIÓN GENERAL DEL DATAFRAME \n')
    print(df.info())
    print('____________________________________________________________\n')
    
    print('Ver los NULOS del DataFrame \n')
    print(f'Nulos de todo el data: --> {df.isnull().sum().mean() * 100} \n')
    for columna in df.columns:
        cantidad_valores_nulos = df[columna].isnull().mean() * 100
        print(f'La columna {columna}: {cantidad_valores_nulos}')
    print('____________________________________________________________\n')
    
    print('Valores ÚNICOS por columna:\n')
    for columna in df.columns:
        cantidad_valores_unicos = df[columna].unique()
        print(f'La columna {columna}: {len(cantidad_valores_unicos)}')
        print(f'La columna {columna}: {cantidad_valores_unicos}')
        
    print('____________________________________________________________\n')
    
    print('Valores DUPLICADOS por columna es de:\n')
    for columna in df.columns:
        cantidad_duplicados = df[columna].duplicated().sum()
        print(f'La columna {columna}: {cantidad_duplicados}')
    print('____________________________________________________________\n')
  
    print('--> RESUMEN ESTADÍSTICO \n')
    print('<<< Variables Numéricas >>> \n')
    print(df.describe().T)
    print('<<< Variables Categóricas >>> \n')
    print(df.describe(include='object').T)
   

In [15]:
exploracion_datos1(df_loyal)

 Filas y Columnas del DATAFRAME 

El número de filas que tenemos es de 16737.
El número de columnas es de 16

____________________________________________________________

 Nombre de todas las Columnas del DATAFRAME: 

Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month'],
      dtype='object')
____________________________________________________________

INFORMACIÓN GENERAL DEL DATAFRAME 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null

## --> Union de los dos csv

In [16]:
print(df_fly.columns)
print(df_loyal.columns)

Index(['Loyalty Number', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')
Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month'],
      dtype='object')


In [17]:
df_merge_left = df_fly.merge(df_loyal, how = 'left', left_on = 'Loyalty Number', right_on = 'Loyalty Number')
df_merge_left.sample(5)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
189157,482561,2017,10,2,2,4,1672,167.0,386,31,Canada,Alberta,Edmonton,T3G 6Y6,Male,Bachelor,68249.0,Single,Star,8769.27,Standard,2013,7,NaN,NaN
54364,296736,2017,4,0,0,0,0,0.0,0,0,Canada,Quebec,Montreal,H2Y 2W2,Female,College,NaN,Married,Star,4717.63,Standard,2016,5,NaN,NaN
178581,609072,2017,11,10,0,10,2460,246.0,0,0,Canada,Quebec,Montreal,H2T 9K8,Female,Doctor,117643.0,Married,Star,2619.17,Standard,2017,4,NaN,NaN
207877,371159,2018,1,0,0,0,0,0.0,0,0,Canada,Ontario,Trenton,K8V 4B2,Female,College,NaN,Single,Nova,6559.35,Standard,2014,12,NaN,NaN
237935,618523,2018,1,1,0,1,1306,130.0,0,0,Canada,Ontario,Trenton,K8V 4B2,Male,Doctor,87499.0,Married,Aurora,9939.26,Standard,2015,2,2018.0,12.0


In [18]:
print(f"las filas y columnas que tenemos en df_fly: {df_fly.shape}")
print("-------------------------------------------------------------------")

print(f"las filas y columnas que tenemos en df_loyal: {df_loyal.shape}")
print("-------------------------------------------------------------------")

print(f"las filas y columnas que tenemos ahora en la unión de las dos tablas: {df_merge_left.shape}")


las filas y columnas que tenemos en df_fly: (405624, 10)
-------------------------------------------------------------------
las filas y columnas que tenemos en df_loyal: (16737, 16)
-------------------------------------------------------------------
las filas y columnas que tenemos ahora en la unión de las dos tablas: (405624, 25)


In [19]:
# Identificar filas duplicadas exactas (todas las columnas iguales)
exact_duplicates = df_merge_left[df_merge_left.duplicated(keep=False)]

# Filtrar los duplicados que también tienen el mismo 'Loyalty Number'
exact_duplicates_sorted = exact_duplicates.sort_values(by=['Loyalty Number', 'Year', 'Month'])

# Mostrar el DataFrame resultante
exact_duplicates_sorted.sample(5)


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
57596,467894,2017,4,0,0,0,0,0.0,0,0,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,Doctor,116909.0,Married,Nova,20725.38,Standard,2015,2,NaN,NaN
349188,693926,2018,9,0,0,0,0,0.0,0,0,Canada,Ontario,Toronto,M2Z 4K1,Male,High School or Below,48104.0,Married,Star,2665.22,Standard,2013,3,NaN,NaN
324160,263267,2018,8,0,0,0,0,0.0,0,0,Canada,Quebec,Montreal,H2Y 2W2,Male,College,NaN,Single,Nova,11624.82,Standard,2018,11,NaN,NaN
371082,960050,2018,10,0,0,0,0,0.0,0,0,Canada,Ontario,Thunder Bay,K8T 5M5,Female,Bachelor,103151.0,Single,Aurora,8622.84,Standard,2015,12,NaN,NaN
218972,960050,2018,1,0,0,0,0,0.0,0,0,Canada,Ontario,Thunder Bay,K8T 5M5,Female,Bachelor,103151.0,Single,Aurora,8622.84,Standard,2015,12,NaN,NaN


In [20]:
# Eliminar duplicados en la columna 'employeenumber' y conservar solo la primera ocurrencia
df_merge_left.drop_duplicates(keep='first', inplace=True)

In [21]:
df_merge_left.shape

(403760, 25)

In [22]:
# Guarda el DataFrame como un archivo CSV
df_merge_left.to_csv("DF_sin_duplicados.csv", index = False)